In [2]:
"""
Building Modifications Analysis - Jupyter Notebook Code

This code analyzes building modification data where multiple variants are compared.
It creates tabular views with variants as columns to easily compare changes.

USAGE:
1. Update the 'filepath' variable below with your actual file path
2. Run all cells to generate various analysis views
3. Use the provided functions to filter and analyze specific categories
"""

import pandas as pd
import json
import numpy as np
from IPython.display import display, HTML
import os

# Read the modifications file (handles both Parquet and JSON lines)
def read_modifications_file(filepath):
    """Read modifications file - supports both Parquet and JSON lines formats"""
    if filepath.endswith('.parquet'):
        return pd.read_parquet(filepath)
    else:
        # JSON lines format
        data = []
        with open(filepath, 'r') as f:
            for line in f:
                data.append(json.loads(line.strip()))
        return pd.DataFrame(data)

# Load your data - UPDATE THIS PATH to your actual file location
# Example for Parquet file:
filepath = r'D:\Documents\daily\E_Plus_2040_py\output\f89bc1a7-176a-4f2e-9012-b626392ae859\modified_idfs\modifications_detail_20250626_124705.parquet'

# Or if you saved the JSON data to a file:
# filepath = 'your_modifications.json'

# Check if file exists
if not os.path.exists(filepath):
    print(f"File not found: {filepath}")
    print("Please update the filepath variable with your actual file path")
    raise FileNotFoundError(f"Please update filepath to point to your actual file")

df = read_modifications_file(filepath)

# Display basic info
print(f"Total modifications: {len(df)}")
print(f"Unique variants: {df['variant_id'].nunique()} - {list(df['variant_id'].unique())}")
print(f"Unique categories: {df['category'].nunique()} - {list(df['category'].unique())}")
print(f"Unique buildings: {df['building_id'].nunique()}")
print(f"Data shape: {df.shape}")
print("\n")

# Show sample of raw data
print("Sample of raw data:")
display(df.head())
print("\n")

# Create a unique identifier for each modification
df['modification_id'] = (df['category'] + '_' + 
                        df['object_type'] + '_' + 
                        df['object_name'] + '_' + 
                        df['field_name'])

# Create comparison view: Variants as columns showing new values
def create_variant_comparison(df, value_type='new_value'):
    """Create a pivot table with variants as columns"""
    pivot = df.pivot_table(
        index='modification_id',
        columns='variant_id',
        values=value_type,
        aggfunc='first'
    )
    
    # Add original value column (same for all variants)
    original_values = df.drop_duplicates('modification_id').set_index('modification_id')['original_value']
    pivot.insert(0, 'original_value', original_values)
    
    return pivot

# Create the main comparison table
comparison_df = create_variant_comparison(df)
print("Comparison Table (Original vs Variants):")
print("-" * 80)
display(comparison_df.head(10))

# Create a more detailed view grouped by category
def create_detailed_comparison(df):
    """Create a detailed comparison with better formatting"""
    # Split the modification_id back to components for better display
    result = []
    
    for category in df['category'].unique():
        cat_df = df[df['category'] == category]
        
        for idx, row_data in cat_df.groupby(['object_type', 'object_name', 'field_name']):
            row_dict = {
                'Category': category,
                'Object Type': idx[0],
                'Object Name': idx[1],
                'Field': idx[2],
                'Original': row_data['original_value'].iloc[0]
            }
            
            # Add variant values
            for variant in sorted(df['variant_id'].unique()):
                variant_data = row_data[row_data['variant_id'] == variant]
                if not variant_data.empty:
                    row_dict[variant] = variant_data['new_value'].iloc[0]
                else:
                    row_dict[variant] = np.nan
            
            result.append(row_dict)
    
    return pd.DataFrame(result)

# Create detailed comparison
detailed_df = create_detailed_comparison(df)
print("\n\nDetailed Comparison by Category:")
print("-" * 80)

# For large datasets, show summary first
print(f"Total rows in detailed comparison: {len(detailed_df)}")
print(f"Categories: {detailed_df['Category'].value_counts().to_dict()}")
print("\n")

# Display by category (limit rows for large datasets)
max_rows_per_category = 10  # Adjust this to see more/fewer rows

for category in detailed_df['Category'].unique():
    cat_data = detailed_df[detailed_df['Category'] == category]
    print(f"\n{category.upper()} (showing first {min(max_rows_per_category, len(cat_data))} of {len(cat_data)} modifications):")
    display(cat_data.drop('Category', axis=1).head(max_rows_per_category))

# Calculate change percentages
def calculate_changes(df):
    """Calculate percentage changes from original values"""
    changes_df = detailed_df.copy()
    
    # Get variant columns
    variant_cols = [col for col in changes_df.columns if col.startswith('variant_')]
    
    for var_col in variant_cols:
        # Calculate percentage change where possible
        changes_df[f'{var_col}_change_%'] = pd.NA
        
        # For numeric values, calculate percentage change
        numeric_mask = pd.to_numeric(changes_df['Original'], errors='coerce').notna()
        
        if numeric_mask.any():
            original_numeric = pd.to_numeric(changes_df.loc[numeric_mask, 'Original'], errors='coerce')
            new_numeric = pd.to_numeric(changes_df.loc[numeric_mask, var_col], errors='coerce')
            
            changes_df.loc[numeric_mask, f'{var_col}_change_%'] = (
                ((new_numeric - original_numeric) / original_numeric * 100).round(2)
            )
    
    return changes_df

# Calculate changes
changes_df = calculate_changes(df)

# Display changes for numeric fields only
numeric_changes = changes_df[pd.to_numeric(changes_df['Original'], errors='coerce').notna()]
print("\n\nPercentage Changes from Original Values:")
print("-" * 80)

# Show only the percentage change columns (limit rows for large datasets)
change_cols = ['Category', 'Object Type', 'Object Name', 'Field', 'Original'] + \
              [col for col in numeric_changes.columns if 'change_%' in col]
display(numeric_changes[change_cols].head(15))

# Compact summary view for all modifications
def create_compact_summary():
    """Create a compact view showing all modifications grouped by category"""
    summary_data = []
    
    for category in df['category'].unique():
        cat_df = df[df['category'] == category]
        
        # Get unique modifications for this category
        mods = cat_df.groupby(['object_name', 'field_name']).agg({
            'original_value': 'first',
            'variant_id': 'count'
        }).reset_index()
        
        # Get variant values
        for _, mod in mods.iterrows():
            row = {
                'Category': category,
                'Object': mod['object_name'],
                'Field': mod['field_name'],
                'Original': mod['original_value']
            }
            
            # Add variant values
            for variant in sorted(df['variant_id'].unique()):
                var_val = cat_df[
                    (cat_df['object_name'] == mod['object_name']) & 
                    (cat_df['field_name'] == mod['field_name']) & 
                    (cat_df['variant_id'] == variant)
                ]['new_value'].values
                
                if len(var_val) > 0:
                    row[variant] = var_val[0]
            
            summary_data.append(row)
    
    return pd.DataFrame(summary_data)

print("\n\nCompact Summary View:")
print("-" * 80)
compact_df = create_compact_summary()
print(f"Total unique modifications across all variants: {len(compact_df)}")

# Show first few from each category
for cat in compact_df['Category'].unique():
    cat_data = compact_df[compact_df['Category'] == cat]
    print(f"\n{cat.upper()} ({len(cat_data)} modifications):")
    display(cat_data.head(5))

# Summary statistics by category
def create_summary_stats(df):
    """Create summary statistics of changes by category"""
    summary = []
    
    for category in df['category'].unique():
        cat_df = df[df['category'] == category]
        
        for variant in df['variant_id'].unique():
            var_df = cat_df[cat_df['variant_id'] == variant]
            
            summary.append({
                'Category': category,
                'Variant': variant,
                'Number of Changes': len(var_df),
                'Unique Objects': var_df['object_name'].nunique(),
                'Unique Fields': var_df['field_name'].nunique()
            })
    
    return pd.DataFrame(summary)

# Create and display summary
summary_df = create_summary_stats(df)
summary_pivot = summary_df.pivot_table(
    index='Category',
    columns='Variant',
    values='Number of Changes',
    aggfunc='first'
)

print("\n\nSummary: Number of Changes per Category per Variant:")
print("-" * 80)
display(summary_pivot)

# Export functions
def export_to_excel(comparison_df, detailed_df, changes_df, summary_pivot, filename='building_modifications_analysis.xlsx'):
    """Export all analysis to Excel with multiple sheets"""
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        comparison_df.to_excel(writer, sheet_name='Value Comparison')
        detailed_df.to_excel(writer, sheet_name='Detailed View', index=False)
        changes_df.to_excel(writer, sheet_name='Percentage Changes', index=False)
        summary_pivot.to_excel(writer, sheet_name='Summary')
    
    print(f"\nAnalysis exported to {filename}")

# Uncomment to export
# export_to_excel(comparison_df, detailed_df, changes_df, summary_pivot)

# Interactive filtering function
def filter_by_category(category):
    """Filter and display changes for a specific category"""
    filtered = detailed_df[detailed_df['Category'] == category]
    display(filtered.drop('Category', axis=1))
    
# Example usage:
# filter_by_category('lighting')

# Find differences between variants
def find_variant_differences():
    """Find fields where variants have different values"""
    variant_cols = [col for col in detailed_df.columns if col.startswith('variant_')]
    
    if len(variant_cols) > 1:
        # Check where values differ between variants
        detailed_df['variants_differ'] = False
        
        for i in range(len(variant_cols)-1):
            for j in range(i+1, len(variant_cols)):
                detailed_df['variants_differ'] |= (
                    detailed_df[variant_cols[i]] != detailed_df[variant_cols[j]]
                )
        
        different = detailed_df[detailed_df['variants_differ']]
        print(f"\n\nFields where variants have different values ({len(different)} total):")
        print("-" * 80)
        display(different.drop('variants_differ', axis=1).head(20))

find_variant_differences()

# Create a heatmap-style HTML table for better visualization
def create_html_comparison(df, category=None):
    """Create an HTML table with color-coded changes"""
    if category:
        data = detailed_df[detailed_df['Category'] == category].copy()
    else:
        data = detailed_df.head(30).copy()
    
    # Calculate percentage changes for coloring
    variant_cols = [col for col in data.columns if col.startswith('variant_')]
    
    html = '<table style="border-collapse: collapse; font-size: 12px;">'
    html += '<thead><tr style="background-color: #f0f0f0;">'
    
    # Headers
    for col in ['Object Type', 'Object Name', 'Field', 'Original'] + variant_cols:
        html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{col}</th>'
    html += '</tr></thead><tbody>'
    
    # Rows
    for _, row in data.iterrows():
        html += '<tr>'
        
        # Basic info columns
        for col in ['Object Type', 'Object Name', 'Field']:
            html += f'<td style="border: 1px solid #ddd; padding: 8px;">{row[col]}</td>'
        
        # Original value
        html += f'<td style="border: 1px solid #ddd; padding: 8px; background-color: #e8e8e8;">{row["Original"]}</td>'
        
        # Variant values with color coding
        for var_col in variant_cols:
            value = row[var_col]
            
            # Try to calculate percentage change for color
            try:
                orig_num = float(row['Original'])
                new_num = float(value)
                pct_change = ((new_num - orig_num) / orig_num) * 100
                
                # Color based on change magnitude
                if pct_change > 20:
                    color = '#ffcccc'  # Light red for large increase
                elif pct_change > 0:
                    color = '#ffe6cc'  # Light orange for increase
                elif pct_change < -20:
                    color = '#ccccff'  # Light blue for large decrease
                elif pct_change < 0:
                    color = '#cce6ff'  # Light cyan for decrease
                else:
                    color = 'white'
                    
                cell_text = f'{value}<br><small>({pct_change:+.1f}%)</small>'
            except:
                color = '#f9f9f9'
                cell_text = str(value)
            
            html += f'<td style="border: 1px solid #ddd; padding: 8px; background-color: {color};">{cell_text}</td>'
        
        html += '</tr>'
    
    html += '</tbody></table>'
    
    return HTML(html)

# Display HTML table for a specific category
print("\n\nInteractive HTML Table (Materials category):")
display(create_html_comparison(df, 'materials'))

# Quick analysis function for large datasets
def quick_analysis_by_category(category, show_differences_only=False):
    """Quick analysis of a specific category with option to show only differences between variants"""
    cat_df = df[df['category'] == category]
    
    print(f"\nAnalysis for {category.upper()}:")
    print(f"Total modifications: {len(cat_df)}")
    print(f"Unique objects: {cat_df['object_name'].nunique()}")
    print(f"Unique fields modified: {cat_df['field_name'].nunique()}")
    
    # Create pivot for this category
    pivot = cat_df.pivot_table(
        index=['object_name', 'field_name'],
        columns='variant_id',
        values='new_value',
        aggfunc='first'
    )
    
    # Add original values
    orig_vals = cat_df.drop_duplicates(['object_name', 'field_name']).set_index(['object_name', 'field_name'])['original_value']
    pivot.insert(0, 'original', orig_vals)
    
    if show_differences_only and len(pivot.columns) > 2:
        # Filter to show only rows where variants differ
        variant_cols = [col for col in pivot.columns if col.startswith('variant_')]
        mask = False
        for i in range(len(variant_cols)-1):
            mask |= (pivot[variant_cols[i]] != pivot[variant_cols[i+1]])
        pivot = pivot[mask]
        print(f"\nShowing {len(pivot)} modifications where variants differ:")
    
    display(pivot)
    return pivot

# Example: Analyze specific categories
# quick_analysis_by_category('materials', show_differences_only=True)
# quick_analysis_by_category('lighting')
# quick_analysis_by_category('shading', show_differences_only=True)

Total modifications: 893
Unique variants: 19 - ['variant_0', 'variant_1', 'variant_2', 'variant_3', 'variant_4', 'variant_5', 'variant_6', 'variant_7', 'variant_8', 'variant_9', 'variant_10', 'variant_11', 'variant_12', 'variant_13', 'variant_14', 'variant_15', 'variant_16', 'variant_17', 'variant_18']
Unique categories: 9 - ['lighting', 'materials', 'ventilation', 'equipment', 'dhw', 'geometry', 'shading', 'simulation_control', 'site_location']
Unique buildings: 1
Data shape: (893, 10)


Sample of raw data:


,building_id,variant_id,category,object_type,object_name,field_name,original_value,new_value,change_type,timestamp
0,4136733,variant_0,lighting,LIGHTS,Lights_ALL_ZONES,fraction_radiant,0.1,0.2,absolute,2025-06-26T12:47:05.973534
1,4136733,variant_0,lighting,LIGHTS,Lights_ALL_ZONES,fraction_visible,0.1,0.2,absolute,2025-06-26T12:47:05.973546
2,4136733,variant_0,lighting,LIGHTS,Lights_ALL_ZONES,return_air_fraction,0.8,0.55,absolute,2025-06-26T12:47:05.973549
3,4136733,variant_0,materials,MATERIAL,Concrete_200mm,conductivity,0.0781191888777523,0.05616695099421968,absolute,2025-06-26T12:47:05.973552
4,4136733,variant_0,materials,MATERIAL,Concrete_200mm,thickness,0.195298,0.2748610163429042,absolute,2025-06-26T12:47:05.973555




Comparison Table (Original vs Variants):
--------------------------------------------------------------------------------


variant_id,original_value,variant_0,variant_1,variant_10,variant_11,variant_12,variant_13,variant_14,variant_15,variant_16,variant_17,variant_18,variant_2,variant_3,variant_4,variant_5,variant_6,variant_7,variant_8,variant_9
modification_id,,,,,,,,,,,,,,,,,,,,
dhw_WATERHEATER:MIXED_MyDHW_0_WaterHeater_heater_efficiency,0.9,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99
dhw_WATERHEATER:MIXED_MyDHW_0_WaterHeater_off_cycle_loss_coefficient,5.0,3.9507952627282976,3.9520183453271756,3.761514189871616,3.627735479696377,3.43965185459688,3.282741351090612,3.5483400849008557,3.824880779994592,3.2696288023089637,3.4349017810601485,3.231897046409198,3.558374692529828,3.308500296267921,3.173720133493558,3.827195072371862,3.771067508567603,3.9860534913797214,3.9137488721748626,3.215341007230845
equipment_ELECTRICEQUIPMENT_Equip_ALL_ZONES_watts_per_area,5.134624316298921,3.7097452188846813,4.016396804454633,3.5076577633477464,2.621387269611836,3.7548024457748324,3.725538198807911,4.01950857444077,3.1720714421915273,3.1246585559131557,4.104414125105223,3.72561875461157,3.0951988153830365,3.4160855576700113,2.6568115031270945,2.8715216997853057,2.6535514060276344,4.066800533155424,3.684344577548048,3.1121023009490036
geometry_FENESTRATIONSURFACE:DETAILED_Zone1_FrontPerimeter_Wall_0 window_window_multiplier,1.0,1.2021150461448094,1.1869263480766385,1.2578105608203187,1.25598927446702,1.222390248820119,1.0173880858957118,1.1461813384638824,1.2452180408166547,1.1500429151442337,1.010394150471498,1.147132224984872,1.1952991225090628,1.2207357190489587,1.0744827931791259,1.1913841051107634,1.265115229123687,1.2088473113972926,1.2937483954542208,1.1638317497111252
geometry_FENESTRATIONSURFACE:DETAILED_Zone1_LeftPerimeter_Wall_0 window_window_multiplier,1.0,1.1309095152687916,1.0538320644181463,1.207178280159359,1.282082373603501,1.0621796608849263,1.2094776375176974,1.126685989827209,1.1441480197373954,1.2600120973643856,1.045198128988161,1.1192914521957094,1.0252681641445536,1.0933711156558088,1.0447114101787698,1.097273619454542,1.009016607801172,1.0041214430226972,1.2334990857756916,1.1170592165114464
geometry_FENESTRATIONSURFACE:DETAILED_Zone1_RearPerimeter_Wall_0 window_window_multiplier,1.0,1.245011632173943,1.0367468717903252,1.0854764245829363,1.2436704499279465,1.2234940964543526,1.086424888070757,1.26390195665855,1.2976423252724465,1.187957604173752,1.143170285734674,1.263830032338821,1.2735889236110525,1.0015304401526375,1.0823973245091518,1.199377406409064,1.1599886927469156,1.1200311470115218,1.167657299178195,1.0000143214615433
geometry_FENESTRATIONSURFACE:DETAILED_Zone2_FrontPerimeter_Wall_0 window_window_multiplier,1.0,1.0113500574348975,1.192734560523533,1.288019261726181,1.0243877228231526,1.021464343233174,1.2248972118443662,1.130597075243883,1.0272800706847252,1.0945106062300762,1.2987977679221583,1.0525813800631456,1.270687927103382,1.160707312890597,1.0790102283055156,1.2131384014408382,1.003070671984728,1.0740976092145662,1.0721240764987319,1.1923869282778863
geometry_FENESTRATIONSURFACE:DETAILED_Zone2_LeftPerimeter_Wall_0 window_window_multiplier,1.0,1.2211138051251822,1.1858911230774891,1.2629981171309106,1.1206027840986803,1.2000396487513902,1.0710665176366734,1.204961760525503,1.0237395238188332,1.0502424909283896,1.2423768098501564,1.0087114281501766,1.0835784034707823,1.0388072732833462,1.2390632223264444,1.1019000964842283,1.2188549665169066,1.170631655273755,1.2118909192080731,1.0557657653893187
geometry_FENESTRATIONSURFACE:DETAILED_Zone2_RearPerimeter_Wall_0 window_window_multiplier,1.0,1.2140739940733847,1.2506664671124448,1.0298448756506853,1.0114376590728993,1.2341968360366893,1.237081014039315,1.0712829695454074,1.206515594451634,1.1805416106737612,1.1679374121818842,1.0836400518952662,1.2312086749080227,1.0635607733006847,1.121377717930779,1.2626489607810565,1.0047302897905943,1.057750219310922,1.2522330475598586,1.0774021975956771




Detailed Comparison by Category:
--------------------------------------------------------------------------------
Total rows in detailed comparison: 47
Categories: {'site_location': 13, 'shading': 12, 'materials': 7, 'geometry': 6, 'lighting': 3, 'ventilation': 2, 'dhw': 2, 'equipment': 1, 'simulation_control': 1}



LIGHTING (showing first 3 of 3 modifications):


...

Variant,variant_0,variant_1,variant_10,variant_11,variant_12,variant_13,variant_14,variant_15,variant_16,variant_17,variant_18,variant_2,variant_3,variant_4,variant_5,variant_6,variant_7,variant_8,variant_9
Category,,,,,,,,,,,,,,,,,,,
dhw,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
equipment,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
geometry,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
lighting,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
materials,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
shading,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
simulation_control,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
site_location,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
ventilation,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2




Fields where variants have different values (34 total):
--------------------------------------------------------------------------------


,Category,Object Type,Object Name,Field,Original,variant_0,variant_1,variant_10,variant_11,variant_12,...,variant_17,variant_18,variant_2,variant_3,variant_4,variant_5,variant_6,variant_7,variant_8,variant_9
3,materials,MATERIAL,Concrete_200mm,conductivity,0.0781191888777523,0.05616695099421968,0.05181246873710996,0.05574215808293488,0.05164326721843373,0.05328706466590547,...,0.05057058129612405,0.05125393364701943,0.04951702689216031,0.04774900858094279,0.05262297350366127,0.0547408112708013,0.05985452696507199,0.04801203393127346,0.06183867068029065,0.056245045010844645
4,materials,MATERIAL,Concrete_200mm,thickness,0.195298,0.2748610163429042,0.25462410496546817,0.23939073982892436,0.23702662072770775,0.2526937076261072,...,0.26597266460991004,0.26569351948198033,0.25072594802538883,0.2490097496503727,0.27413197103823334,0.25649982247566644,0.2810855208386,0.26897045344307496,0.27903542016400257,0.2648059034496688
5,materials,MATERIAL:NOMASS,Door_U3.4,thermal_resistance,0.29,0.4512414249337989,0.4673562431935747,0.5269075624485514,0.4432148154711362,0.5323205987579149,...,0.5740393529618117,0.4834899648941118,0.5604163574003795,0.48218745229716237,0.5296682561568926,0.5563401932465057,0.45850694018504157,0.45133813227572495,0.5411160985459806,0.565938635502737
6,materials,MATERIAL:NOMASS,GroundFloor_R2.5,thermal_resistance,0.3448275862068966,0.6236586130886351,0.5443567679993165,0.5371963810345735,0.576395886872114,0.5335278597830054,...,0.5749551121320563,0.6863065930019216,0.6055576787825696,0.5722595018652876,0.5722045885307144,0.5505681760503426,0.5832827979133323,0.6091202761836183,0.5700788919806306,0.5318081160391621
7,materials,MATERIAL:NOMASS,InteriorWall_Generic_R0.5,thermal_resistance,0.5,0.9603807242265523,0.7609057971797316,0.9385848309816311,0.7526829994804287,0.7578823886991464,...,0.7683657123106723,0.9902045300534033,0.7662190794881039,0.8818929942564271,0.8984285182079081,0.8196529850061978,0.7682518312890189,0.9958220647213578,0.9906213027948376,0.7614488970074267
8,materials,MATERIAL:NOMASS,InternalFloor_Generic_R1.0,thermal_resistance,1.0,1.536776715289134,1.9197080982387595,1.7050834556330159,1.627342118433412,1.6333948922463235,...,1.8210649034766815,1.9304984696682421,1.6312820910423291,1.7985885305917393,1.6609153432469834,1.9010204176985543,1.9718026991658306,1.96284166058253,1.8290992308661678,1.5389956048935565
9,materials,MATERIAL:NOMASS,Roof_R2.5,thermal_resistance,2.5,4.157649840771075,4.774517444645493,4.124293644126075,3.9079551783325823,4.171881237113723,...,3.854972072558811,4.401180727955215,4.951163623277891,4.03968592104966,4.611313669631031,4.3220114345867415,4.346566042383525,4.59113162003261,4.641945227963957,4.215571128513178
12,equipment,ELECTRICEQUIPMENT,Equip_ALL_ZONES,watts_per_area,5.134624316298921,3.7097452188846813,4.016396804454633,3.5076577633477464,2.621387269611836,3.7548024457748324,...,4.104414125105223,3.72561875461157,3.0951988153830365,3.4160855576700113,2.6568115031270945,2.8715216997853057,2.6535514060276344,4.066800533155424,3.684344577548048,3.1121023009490036
14,dhw,WATERHEATER:MIXED,MyDHW_0_WaterHeater,off_cycle_loss_coefficient,5.0,3.9507952627282976,3.9520183453271756,3.761514189871616,3.627735479696377,3.43965185459688,...,3.4349017810601485,3.231897046409198,3.558374692529828,3.308500296267921,3.173720133493558,3.827195072371862,3.771067508567603,3.9860534913797214,3.9137488721748626,3.215341007230845
15,geometry,FENESTRATIONSURFACE:DETAILED,Zone1_FrontPerimeter_Wall_0 window,window_multiplier,1.0,1.2021150461448094,1.1869263480766385,1.2578105608203187,1.25598927446702,1.222390248820119,...,1.010394150471498,1.147132224984872,1.1952991225090628,1.2207357190489587,1.0744827931791259,1.1913841051107634,1.265115229123687,1.2088473113972926,1.2937483954542208,1.1638317497111252




Interactive HTML Table (Materials category):


Object Type,Object Name,Field,Original,variant_0,variant_1,variant_10,variant_11,variant_12,variant_13,variant_14,variant_15,variant_16,variant_17,variant_18,variant_2,variant_3,variant_4,variant_5,variant_6,variant_7,variant_8,variant_9
MATERIAL,Concrete_200mm,conductivity,0.0781191888777523,0.05616695099421968(-28.1%),0.05181246873710996(-33.7%),0.05574215808293488(-28.6%),0.05164326721843373(-33.9%),0.05328706466590547(-31.8%),0.058841191126213964(-24.7%),0.06022722617368322(-22.9%),0.054451163270672495(-30.3%),0.057201364112560917(-26.8%),0.05057058129612405(-35.3%),0.05125393364701943(-34.4%),0.04951702689216031(-36.6%),0.04774900858094279(-38.9%),0.05262297350366127(-32.6%),0.0547408112708013(-29.9%),0.05985452696507199(-23.4%),0.04801203393127346(-38.5%),0.06183867068029065(-20.8%),0.056245045010844645(-28.0%)
MATERIAL,Concrete_200mm,thickness,0.195298,0.2748610163429042(+40.7%),0.25462410496546817(+30.4%),0.23939073982892436(+22.6%),0.23702662072770775(+21.4%),0.2526937076261072(+29.4%),0.24301619185475196(+24.4%),0.2827389518103392(+44.8%),0.27841149017766165(+42.6%),0.2575607354616413(+31.9%),0.26597266460991004(+36.2%),0.26569351948198033(+36.0%),0.25072594802538883(+28.4%),0.2490097496503727(+27.5%),0.27413197103823334(+40.4%),0.25649982247566644(+31.3%),0.2810855208386(+43.9%),0.26897045344307496(+37.7%),0.27903542016400257(+42.9%),0.2648059034496688(+35.6%)
MATERIAL:NOMASS,Door_U3.4,thermal_resistance,0.29,0.4512414249337989(+55.6%),0.4673562431935747(+61.2%),0.5269075624485514(+81.7%),0.4432148154711362(+52.8%),0.5323205987579149(+83.6%),0.5562143390688034(+91.8%),0.5756437062530199(+98.5%),0.5592282794167907(+92.8%),0.5575006864730568(+92.2%),0.5740393529618117(+97.9%),0.4834899648941118(+66.7%),0.5604163574003795(+93.2%),0.48218745229716237(+66.3%),0.5296682561568926(+82.6%),0.5563401932465057(+91.8%),0.45850694018504157(+58.1%),0.45133813227572495(+55.6%),0.5411160985459806(+86.6%),0.565938635502737(+95.2%)
MATERIAL:NOMASS,GroundFloor_R2.5,thermal_resistance,0.3448275862068966,0.6236586130886351(+80.9%),0.5443567679993165(+57.9%),0.5371963810345735(+55.8%),0.576395886872114(+67.2%),0.5335278597830054(+54.7%),0.5368164512230011(+55.7%),0.5815932013400039(+68.7%),0.6245947366029972(+81.1%),0.5966161377387218(+73.0%),0.5749551121320563(+66.7%),0.6863065930019216(+99.0%),0.6055576787825696(+75.6%),0.5722595018652876(+66.0%),0.5722045885307144(+65.9%),0.5505681760503426(+59.7%),0.5832827979133323(+69.2%),0.6091202761836183(+76.6%),0.5700788919806306(+65.3%),0.5318081160391621(+54.2%)
MATERIAL:NOMASS,InteriorWall_Generic_R0.5,thermal_resistance,0.5,0.9603807242265523(+92.1%),0.7609057971797316(+52.2%),0.9385848309816311(+87.7%),0.7526829994804287(+50.5%),0.7578823886991464(+51.6%),0.7673838104281068(+53.5%),0.8245528593339976(+64.9%),0.9969870523386555(+99.4%),0.7759976589360675(+55.2%),0.7683657123106723(+53.7%),0.9902045300534033(+98.0%),0.7662190794881039(+53.2%),0.8818929942564271(+76.4%),0.8984285182079081(+79.7%),0.8196529850061978(+63.9%),0.7682518312890189(+53.7%),0.9958220647213578(+99.2%),0.9906213027948376(+98.1%),0.7614488970074267(+52.3%)
MATERIAL:NOMASS,InternalFloor_Generic_R1.0,thermal_resistance,1.0,1.536776715289134(+53.7%),1.9197080982387595(+92.0%),1.7050834556330159(+70.5%),1.627342118433412(+62.7%),1.6333948922463235(+63.3%),1.8113997166667661(+81.1%),1.5054281015250726(+50.5%),1.9854600410717491(+98.5%),1.912658958838514(+91.3%),1.8210649034766815(+82.1%),1.9304984696682421(+93.0%),1.6312820910423291(+63.1%),1.7985885305917393(+79.9%),1.6609153432469834(+66.1%),1.9010204176985543(+90.1%),1.9718026991658306(+97.2%),1.96284166058253(+96.3%),1.8290992308661678(+82.9%),1.5389956048935565(+53.9%)
MATERIAL:NOMASS,Roof_R2.5,thermal_resistance,2.5,4.157649840771075(+66.3%),4.774517444645493(+91.0%),4.124293644126075(+65.0%),3.9079551783325823(+56.3%),4.171881237113723(+66.9%),4.607891975112207(+84.3%),3.79397120796575(+51.8%),4.622421665789855(+84.9%),4.86039013007487(+94.4%),3.854972072558811(+54.2%